# Install Necessary Library


In [ ]:
!pip install tensorflow==2.10
!pip install deepctr[cpu]
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download mohamedbakhet/amazon-books-reviews
!unzip /content/amazon-books-reviews.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 54.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.10.0 which is incompatible.
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
 99% 1.05G/1.06G [00:09<00:00, 82.6MB/s]
100% 1.06G/1.06G [00:09<00:00, 124MB/s] 
Archive:  /content/amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_d

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score , mean_squared_error

from itertools import chain

import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Concatenate ,Flatten ,Lambda

from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.feature_column import build_input_features, get_linear_logit, DEFAULT_GROUP_NAME, input_from_feature_columns
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.interaction import FM ,InteractingLayer
from deepctr.layers.utils import concat_func, add_func, combined_dnn_input

from deepctr.feature_column import build_input_features, get_linear_logit, input_from_feature_columns ,VarLenSparseFeat

from deepctr.layers.core import PredictionLayer, DNN ,RegulationModule
from deepctr.layers.interaction import CrossNet ,BridgeModule
from deepctr.layers.utils import add_func, combined_dnn_input


In [ ]:
data = pd.read_csv("Books_rating.csv")
data=data.dropna()
data

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
10,0829814000,Wonderful Worship in Smaller Churches,19.40,AZ0IOBU20TBOP,Rev. Pamela Tinnin,8/10,5.0,991440000,Outstanding Resource for Small Church Pastors,"I just finished the book, &quot;Wonderful Wors..."
11,0829814000,Wonderful Worship in Smaller Churches,19.40,A373VVEU6Z9M0N,Dr. Terry W. Dorsett,1/1,5.0,1291766400,Small Churches CAN Have Wonderful Worship,Many small churches feel like they can not hav...
12,0829814000,Wonderful Worship in Smaller Churches,19.40,AGKGOH65VTRR4,"Cynthia L. Lajoy ""Cindy La Joy""",1/1,5.0,1248307200,Not Just for Pastors!,I just finished reading this amazing book and ...
13,0829814000,Wonderful Worship in Smaller Churches,19.40,A3OQWLU31BU1Y,Maxwell Grant,1/1,5.0,1222560000,Small church pastor? This is the book on worship,I hadn't been a small church pastor very long ...
14,0595344550,Whispers of the Wicked Saints,10.95,A3Q12RK71N74LB,Book Reader,7/11,1.0,1117065600,not good,I bought this book because I read some glowing...
...,...,...,...,...,...,...,...,...,...,...
2999953,0786182431,Very Bad Deaths: Library Edition,90.00,A1EC8SNPR56CLU,Denis Dube,0/0,4.0,1285804800,It's the way he writes it,"""Very Bad Death"" is a so so story, but the cha..."
2999954,0786182431,Very Bad Deaths: Library Edition,90.00,A33VKWCAV9QQKC,"Paige E. Steadman ""RuneEnigma""",0/0,5.0,1230249600,"Bad Deaths, Great Book!",Very Bad Deaths was a very great book! Spider ...
2999955,0786182431,Very Bad Deaths: Library Edition,90.00,A2PK3NTC9RMEF4,S. Potter,0/0,3.0,1179705600,Still read it.,Anything by Spider Robinson is worth reading. ...
2999956,0786182431,Very Bad Deaths: Library Edition,90.00,A2D0PY6HIGTYIT,"Adrian in Phoenix ""No Time for Fantasy""",5/8,5.0,1111276800,Not another Callahan story,Great novel! Easy & enjoyable to read straight...


In [ ]:
sparse_features = ["Id", "Price",
                    "User_id", "review/helpfulness", "review/time"]
target = ['review/score']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# 2.count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].max() + 1, embedding_dim=4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model
train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name: test[name].values for name in feature_names}


In [ ]:
def DCN(linear_feature_columns, dnn_feature_columns, cross_num=2, cross_parameterization='vector',
        dnn_hidden_units=(256, 128, 64), l2_reg_linear=1e-5, l2_reg_embedding=1e-5,
        l2_reg_cross=1e-5, l2_reg_dnn=0, seed=1024, dnn_dropout=0, dnn_use_bn=False,
        dnn_activation='relu', task='regression'):

    if len(dnn_hidden_units) == 0 and cross_num == 0:
        raise ValueError("Either hidden_layer or cross layer must > 0")

    features = build_input_features(dnn_feature_columns)
    inputs_list = list(features.values())

    linear_logit = get_linear_logit(features, linear_feature_columns, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear)

    sparse_embedding_list, dense_value_list = input_from_feature_columns(features, dnn_feature_columns,
                                                                         l2_reg_embedding, seed)

    dnn_input = combined_dnn_input(sparse_embedding_list, dense_value_list)

    if len(dnn_hidden_units) > 0 and cross_num > 0:  # Deep & Cross
        deep_out = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout, dnn_use_bn, seed=seed)(dnn_input)
        cross_out = CrossNet(cross_num, parameterization=cross_parameterization, l2_reg=l2_reg_cross)(dnn_input)
        stack_out = Concatenate()([cross_out, deep_out])
        final_logit = Dense(1, use_bias=False)(stack_out)
    elif len(dnn_hidden_units) > 0:  # Only Deep
        deep_out = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout, dnn_use_bn, seed=seed)(dnn_input)
        final_logit = Dense(1, use_bias=False)(deep_out)
    elif cross_num > 0:  # Only Cross
        cross_out = CrossNet(cross_num, parameterization=cross_parameterization, l2_reg=l2_reg_cross)(dnn_input)
        final_logit = Dense(1, use_bias=False)(cross_out)
    else:  # Error
        raise NotImplementedError

    final_logit = add_func([final_logit, linear_logit])
    output = PredictionLayer(task)(final_logit)

    model = Model(inputs=inputs_list, outputs=output)

    return model

In [ ]:
model = DCN(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)
print("test MSE", round(mean_squared_error(
    test[target].values, pred_ans), 4))

CrossNet parameterization: vector
Epoch 1/10
1037/1037 - 50s - loss: 1.5078 - mse: 1.5055 - val_loss: 1.0387 - val_mse: 1.0330
Epoch 2/10
1037/1037 - 45s - loss: 0.4441 - mse: 0.4368 - val_loss: 1.0771 - val_mse: 1.0682
Epoch 3/10
1037/1037 - 54s - loss: 0.2445 - mse: 0.2337 - val_loss: 1.1133 - val_mse: 1.1004
Epoch 4/10
1037/1037 - 51s - loss: 0.1777 - mse: 0.1640 - val_loss: 1.1006 - val_mse: 1.0858
Epoch 5/10
1037/1037 - 48s - loss: 0.1342 - mse: 0.1190 - val_loss: 1.1139 - val_mse: 1.0979
Epoch 6/10
1037/1037 - 45s - loss: 0.1117 - mse: 0.0955 - val_loss: 1.1027 - val_mse: 1.0860
Epoch 7/10
1037/1037 - 45s - loss: 0.1017 - mse: 0.0849 - val_loss: 1.1180 - val_mse: 1.1007
Epoch 8/10
1037/1037 - 42s - loss: 0.0926 - mse: 0.0753 - val_loss: 1.1198 - val_mse: 1.1022
Epoch 9/10
1037/1037 - 55s - loss: 0.0895 - mse: 0.0718 - val_loss: 1.1370 - val_mse: 1.1190
Epoch 10/10
1037/1037 - 45s - loss: 0.0845 - mse: 0.0667 - val_loss: 1.1653 - val_mse: 1.1474
test MSE 1.1333


In [ ]:
def DIFM(linear_feature_columns, dnn_feature_columns,
         att_embedding_size=8, att_head_num=8, att_res=True, dnn_hidden_units=(256, 128, 64),
         l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, seed=1024, dnn_dropout=0,
         dnn_activation='relu', dnn_use_bn=False, task='regression'):


    if not len(dnn_hidden_units) > 0:
        raise ValueError("dnn_hidden_units is null!")

    features = build_input_features(
        linear_feature_columns + dnn_feature_columns)

    sparse_feat_num = len(list(filter(lambda x: isinstance(x, SparseFeat) or isinstance(x, VarLenSparseFeat),
                                      dnn_feature_columns)))
    inputs_list = list(features.values())

    sparse_embedding_list, _ = input_from_feature_columns(features, dnn_feature_columns,
                                                          l2_reg_embedding, seed)

    if not len(sparse_embedding_list) > 0:
        raise ValueError("there are no sparse features")

    att_input = concat_func(sparse_embedding_list, axis=1)
    att_out = InteractingLayer(att_embedding_size, att_head_num, att_res, scaling=True)(att_input)
    att_out = Flatten()(att_out)
    m_vec = Dense(sparse_feat_num, use_bias=False)(att_out)

    dnn_input = combined_dnn_input(sparse_embedding_list, [])
    dnn_output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn, dnn_dropout, dnn_use_bn, seed=seed)(dnn_input)
    m_bit = Dense(sparse_feat_num, use_bias=False)(dnn_output)

    input_aware_factor = add_func([m_vec, m_bit])  # the complete input-aware factor m_x

    linear_logit = get_linear_logit(features, linear_feature_columns, seed=seed, prefix='linear',
                                    l2_reg=l2_reg_linear, sparse_feat_refine_weight=input_aware_factor)

    fm_input = concat_func(sparse_embedding_list, axis=1)
    refined_fm_input = Lambda(lambda x: x[0] * tf.expand_dims(x[1], axis=-1))(
        [fm_input, input_aware_factor])
    fm_logit = FM()(refined_fm_input)

    final_logit = add_func([linear_logit, fm_logit])

    output = PredictionLayer(task)(final_logit)
    model = Model(inputs=inputs_list, outputs=output)
    return model

In [ ]:
model = DIFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse",
              metrics=['accuracy'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)
print("test MSE", round(mean_squared_error(
    test[target].values, pred_ans), 4))

Epoch 1/10
1037/1037 - 66s - loss: 2.1720 - accuracy: 0.0609 - val_loss: 1.0939 - val_accuracy: 0.0639
Epoch 2/10
1037/1037 - 61s - loss: 0.5279 - accuracy: 0.0604 - val_loss: 1.0889 - val_accuracy: 0.0637
Epoch 3/10
1037/1037 - 76s - loss: 0.2570 - accuracy: 0.0592 - val_loss: 1.0892 - val_accuracy: 0.0638
Epoch 4/10
1037/1037 - 53s - loss: 0.1641 - accuracy: 0.0608 - val_loss: 1.0904 - val_accuracy: 0.0639
Epoch 5/10
1037/1037 - 60s - loss: 0.1354 - accuracy: 0.0617 - val_loss: 1.0911 - val_accuracy: 0.0638
Epoch 6/10
1037/1037 - 60s - loss: 0.1252 - accuracy: 0.0623 - val_loss: 1.1110 - val_accuracy: 0.0639
Epoch 7/10
1037/1037 - 72s - loss: 0.1171 - accuracy: 0.0628 - val_loss: 1.0889 - val_accuracy: 0.0640
Epoch 8/10
1037/1037 - 58s - loss: 0.1095 - accuracy: 0.0632 - val_loss: 1.0928 - val_accuracy: 0.0641
Epoch 9/10
1037/1037 - 53s - loss: 0.1029 - accuracy: 0.0636 - val_loss: 1.0897 - val_accuracy: 0.0639
Epoch 10/10
1037/1037 - 58s - loss: 0.0996 - accuracy: 0.0638 - val_loss: